In [1]:
import os
import time
import random
import numpy as np
import pandas as pd
import multiprocessing
from datetime import datetime
from itertools import islice
from collections import defaultdict, namedtuple
from tuplestate import init_from_solvitaire, to_dict, pprint_st
from benchmarking import *
from solver import solve

In [2]:
def get_state(ret):
    deck_json = convert_shootme_to_solvitaire_json(ret)
    return init_from_solvitaire(deck_json)


def prepare_ui_state(ret):
    parsed = parse_winnable(ret)
    game_state = to_dict(get_state(ret))
    
    def card(c):
        ''' UI uses 10 instead of T '''
        return c.replace('t', '10').replace('T', '10')
    
    return {
        'stock': [card(c).lower() for c in game_state['stock']],
        'tableau': [
            [card(c) for c in tab] 
            for tab in game_state['tableau']
        ],
        'waste': game_state['waste'],
        'foundation': game_state['foundation'],
        'moveSeq': parsed['moves'].strip().split(' ')
    }

## Read benchmark output

In [3]:
results = defaultdict(set)

all_solutions = os.listdir('./bench/shootme/')
seeds = list(map(lambda fname: int(fname[:-4]), all_solutions))

def solve_state(ret):
    lines = ret.splitlines()
    result = lines[15]
    if result.startswith('Minimal solution'):
        return "Solved-Min"
    elif result.startswith("Solved"):
        return "Solved"
    elif result.startswith('Impossible'):
        return "Impossible"
    elif result.startswith('Unknown'):
        return "Unknown"

for seed in sorted(seeds):
    with open(f"bench/shootme/{seed}.txt") as f:
        ret = f.read()
        result = solve_state(ret)
        results[result].add(seed)
    
seed_class = {}
for cls in results.keys():
    seeds_cls = results[cls]
    for seed in seeds_cls:
        seed_class[seed] = cls
        
for res, seeds in results.items():
    print(f"{res:12} {len(seeds):8,}")
total = sum(len(s) for s in results.values())
print(('-'*12) + '-' + ('-'*8))
print(f"{'Total':12} {total:8,}")

Solved-Min      7,876
Impossible      1,282
Unknown           522
Solved            320
---------------------
Total          10,000


## Generate suites

In [4]:
# seeds = np.fromiter(seed_class.keys(), dtype=np.int16)
# rand = np.random.RandomState(0)

# def get_suites(size=10, max=30):
#     rand.shuffle(seeds)
#     start = 0
#     suites = 0
#     while suites < max:
#         end = start + size
#         yield list(map(int, seeds[start:end]))
#         start = end
#         suites += 1

# size = 100
# for i, suite in enumerate(get_suites()):
#     with open(f'./bench/suites/{size}/suite_{i:0>4}.txt', 'w') as f:
#         out = ' '.join(map(str,suite))
#         f.write(out)

## Run suites

In [5]:
def map_seeds_to_states(seed_seq):
    for seed in seed_seq:
        with open(f"bench/shootme/{seed}.txt") as f:
            ret = f.read()
            state = get_state(ret)
            yield seed, state

def get_seeds(fname):
    with open(fname) as f:
        return map(int, f.read().strip().split(' '))
    
    
def get_suite_states(fname):
    return list(map_seeds_to_states(get_seeds(fname)))


def get_suite_files(size):
    pref = f'./bench/suites/{size}/'
    ls = sorted(os.listdir(pref))
    if len(ls) == 0:
        raise Exception(f"no suite files in {pref}")
    return [os.path.join(pref, f) for f in ls]

In [6]:
Result = namedtuple('Result', 
    ['seed', 'time', 
     'solved', 'visited', 'msg',
     'seq', 'seqlen', 
     'datetime', 'shootme']
)

def run_solver(seedstate):
    seed, state = seedstate
    print(f"starting seed {seed}")
    start = time.time()
    sol = solve(state, max_states=200_000)
    seq = None
    seqlen = -1
    if sol.solved:
        seq = " ".join(sol.moveseq)
        seqlen = len(seq)
    end = time.time()
    elapsed = end-start
    shootme = seed_class[seed]
    now = datetime.now().isoformat()
    print(f"done solving {seed} after {elapsed:.1f}")
    return Result(
        seed=seed, time=elapsed,
        solved=sol.solved,
        visited=sol.visited,
        msg=sol.msg,
        seq=seq, seqlen=seqlen, 
        datetime=now, shootme=shootme
    )

In [7]:
def timestr():
    now = datetime.now()
    return now.strftime("%Y%m%d-%H%M%S")

def save_results(results, suite_file):
    df = pd.DataFrame(results)
    df.to_csv(f"./bench/results/{suite_file}-{timestr()}.csv")

In [9]:
with multiprocessing.Pool() as pool:
    for suite_file in get_suite_files(size=10):
        print(f"starting {suite_file}")
        states = get_suite_states(suite_file)
        ret = pool.map(run_solver, states)
        _, fname = os.path.split(suite_file)
        fname, _ = os.path.splitext(fname)
        save_results(ret, fname)
        print(f'done {suite_file}')

starting seed 8178
starting seed 3369
starting seed 2964
starting seed 6672
starting seed 9082
starting seed 1188
starting seed 3033
starting seed 6997
starting seed 4412
starting seed 7503
done solving 6672 after 0.0
done solving 6997 after 0.0
done solving 2964 after 0.0
done solving 3369 after 0.0
done solving 1188 after 0.0
done solving 7503 after 0.0
done solving 9082 after 0.0
done solving 8178 after 0.0
done solving 3033 after 0.0
done solving 4412 after 0.0
starting seed 1432
starting seed 2919
starting seed 7718
starting seed 2337
starting seed 4691
starting seed 4021
starting seed 8102
starting seed 4695
starting seed 3410
starting seed 4107
done solving 4107 after 0.0
done solving 4695 after 0.0
done solving 2919 after 0.0
done solving 3410 after 0.0
done solving 4691 after 0.0
done solving 1432 after 0.0
done solving 4021 after 0.0
done solving 2337 after 0.0
done solving 8102 after 0.0
done solving 7718 after 0.0
starting seed 3808
starting seed 5986
starting seed 5367
sta

starting seed 7019
done solving 284 after 0.0
done solving 5425 after 0.0
done solving 224 after 0.0
done solving 301 after 0.0
done solving 1844 after 0.0
done solving 7019 after 0.0
done solving 687 after 0.0
done solving 1449 after 0.0
done solving 4083 after 0.0
done solving 5014 after 0.0
starting seed 8063
starting seed 8571
starting seed 3005
starting seed 9707
starting seed 397
starting seed 6953
starting seed 2296
starting seed 5962
starting seed 9112
starting seed 9753
done solving 8063 after 0.0
done solving 3005 after 0.0
done solving 2296 after 0.0
done solving 8571 after 0.0
done solving 6953 after 0.0
done solving 397 after 0.0
done solving 9753 after 0.0
done solving 5962 after 0.0
done solving 9707 after 0.0
done solving 9112 after 0.0
starting seed 7184
starting seed 8233
starting seed 1430
starting seed 9292
starting seed 8341
starting seed 6423
starting seed 1815
starting seed 3333
starting seed 5048
starting seed 7480
done solving 6423 after 0.0
done solving 3333 a